<a href="https://colab.research.google.com/github/Armandpl/wandb_jetracer/blob/master/wandb_jetracer_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# 🔥 = W&B ➕ PyTorch ➕ Nvidia jetracer

In [22]:
!pip install wandb

Reason for being yanked: Version contained a bug that could cause files to be synced to the wrong run if runs were executed in parallel
     |████████████████████████████████| 1.6MB 7.6MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
  Found existing installation: wandb 0.10.30
    Uninstalling wandb-0.10.30:
      Successfully uninstalled wandb-0.10.30


In [2]:
!wget -O xy_dataset.py https://raw.githubusercontent.com/Armandpl/wandb_jetracer/master/utils/xy_dataset.py
!pip install pytorch-lightning torchmetrics

--2021-05-24 13:25:38--  https://raw.githubusercontent.com/Armandpl/wandb_jetracer/master/utils/xy_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2618 (2.6K) [text/plain]
Saving to: ‘xy_dataset.py’

xy_dataset.py       100%[===================>]   2.56K  --.-KB/s    in 0s      

2021-05-24 13:25:38 (42.2 MB/s) - ‘xy_dataset.py’ saved [2618/2618]

     |████████████████████████████████| 808kB 8.9MB/s 
     |████████████████████████████████| 276kB 19.2MB/s 
     |████████████████████████████████| 829kB 33.6MB/s 
     |████████████████████████████████| 112kB 48.7MB/s 
     |████████████████████████████████| 645kB 34.8MB/s 
     |████████████████████████████████| 1.3MB 46.8MB/s 
     |████████████████████████████████| 296kB 47.6MB/s 
     |██████████

In [1]:
import math
import os

import cv2
import numpy as np
import PIL
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchmetrics
import wandb

from xy_dataset import XYDataset

# 1. Pre-process dataset(s)

In [ ]:
def make_dirs(output_dataset):
    out_dirs = [os.path.join(output_dataset, split) for split in ["train", "val", "test"]]
    [os.makedirs(curr_dir) for curr_dir in out_dirs]

    return out_dirs

In [ ]:
def split_list_by_pct(data, pcts):
    sizes = [int(pct*len(data)) for pct in pcts]

    it = iter(data)
    return [[next(it) for _ in range(size)] for size in sizes]

In [ ]:
import random

# Manually set pytorch seed to get the same dataset split everytime
torch.manual_seed(42)

config = dict(
    datasets=["suzuka:latest", "nurburgring:latest"],
    output_dataset="mix_ready",
    split_pcts=[0.7, 0.2, 0.1],
)

with wandb.init(project="racecar", config=config, entity="wandb", job_type="pre-process-dataset") as run:
    config = run.config

    # TODO: maybe make a tmp dir and delete afterwards, to only keep artifacts in artifacts dir
    # and to be able to run this cell more than once for the same output_dataset value
    out_dirs = make_dirs(config.output_dataset)

    # make sure the train/val/test pct are coherent
    assert math.fsum(config.split_pcts) == 1

    for dataset in config.datasets:
        artifact = run.use_artifact(dataset)
        artifact_dir = artifact.download()
        
        all_fnames = os.listdir(artifact_dir)
        random.shuffle(all_fnames)

        train, val, test = split_list_by_pct(all_fnames, config.split_pcts)
        sets = {
            "train": train,
            "val": val,
            "test": test
        }
        
        for out_dir, split in zip(out_dirs, ["train", "val", "test"]):
            for fname in sets[split]:
                source = os.path.join(artifact_dir, fname)
                dest = os.path.join(out_dir, fname)
                os.rename(source, dest)
    
    # upload artifacct
    artifact = wandb.Artifact(config.output_dataset, type='dataset')
    artifact.add_dir(config.output_dataset)
    run.log_artifact(artifact)

wandb: Adding directory to artifact (./mix_ready)... Done. 0.7s


# 2. Train

In [2]:
def show_label(image, coordinates, color=(0, 255, 0)):
    x, y = coordinates

    x = int((x + 1) / 2 * 224)
    y = int((y + 1) / 2 * 224)

    cv2.circle(image, (x, y), 5, color, 2)

    return image

In [3]:
class RoadRegression(pl.LightningModule):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.mse = torchmetrics.MeanSquaredError()
        self.mae = torchmetrics.MeanAbsoluteError()

        self.model = torchvision.models.__dict__[config.architecture](pretrained=config.pretrained)
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        images, targets = batch
        preds = self.forward(images)
        loss = getattr(F, self.config.loss)(preds, targets)

        # log
        self.log_dict({
            "train/mse": loss,
            "train/mae": self.mae(preds, targets)    
        })

        return loss

    def trainin_epoch_end(self):
        self.mae.reset()
        self.mse.reset()

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        preds = self.forward(images)
        self.mse(preds, targets)
        self.mae(preds, targets)

    def validation_epoch_end(self, val_step_outputs):
        self.log_dict({
            "valid/mse": self.mse.compute(),
            "valid/mae": self.mae.compute()
        })
        self.mse.reset()
        self.mae.reset()
    
    def test_step(self, batch, batch_idx):
        images, targets = batch
        preds = self.forward(images)
        return (images, preds, targets)

    def test_epoch_end(self, test_step_outputs):
        print("test epoch end")
        images, predictions, targets = test_step_outputs[0]
        for i in range(1, len(test_step_outputs)):
            imgs, preds, targs = test_step_outputs[i]

            images = torch.cat((images, imgs), dim=0)
            predictions = torch.cat((predictions, preds), dim=0)
            targets = torch.cat((targets, targs), dim=0)

        # TODO: fetch the loss from config and add other metrics
        losses = F.mse_loss(predictions, targets, reduction='none')

        # display preds and targets on images
        images_with_preds = []
        for idx, image in enumerate(images):
            img = image.permute(1, 2, 0).cpu().numpy()*255

            img = cv2.cvtColor(
                img, cv2.COLOR_BGR2RGB
            )

            img = show_label(img, targets[i])
            img = show_label(img, predictions[i], (0, 0, 255))

            # TODO: check why the colors are weird?
            # because we normalize them in the dataset class?? but then the
            # circles colors are weird too?
            images_with_preds.append(img)

        my_data = [
            [wandb.Image(img), pred, target, loss.sum()] 
            for img, pred, target, loss
            in zip(images_with_preds, predictions, targets, losses)
        ]

        # create a wandb.Table() with corresponding columns
        columns= ["image", "prediction", "target", "loss"]
        test_table = wandb.Table(data=my_data, columns=columns)
        
        artifact = wandb.Artifact("test_predictions", type="test")
        artifact.add(test_table, "test_table")
        wandb.log_artifact(artifact)

        # TODO: use trainer.logger.expriment.log ?
        wandb.log_artifact({"test/examples", test_table})

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.learning_rate)
        return optimizer

In [4]:
def make_loaders(config):
    # Pull the dataset
    artifact = wandb.use_artifact(config.dataset)
    artifact_dir = artifact.download()

    datasets = [XYDataset(os.path.join(artifact_dir, split)) for split in ["train", "val", "test"]]

    # TODO: turn off shuffle for val and test dataloaders
    train, val, test = [make_loader(current_set, config.batch_size) for current_set in datasets]

    return train, val, test

In [5]:
def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

In [6]:
config = dict(
    epochs=5,
    architecture="resnet34",
    pretrained=True,
    batch_size=64,
    learning_rate=1e-4,
    dataset="mix_ready:latest",
    test_dataset=None, # bypasses the dataset test split to use another artifact TODO: implement that
    train_augs=False,
    loss="mse_loss"
    )

with wandb.init(project="racecar", config=config, job_type="train", entity="wandb") as run:
    config = run.config

    train, val, test = make_loaders(config)

    road_regression = RoadRegression(config)
    print(road_regression)

    wandb_logger = WandbLogger()
    trainer = pl.Trainer(logger=wandb_logger, gpus=1, max_epochs=config.epochs)
    trainer.fit(road_regression, train, val)

    trainer.test(test_dataloaders=test)

    # finally we log the models to wandb
    trainer.save_checkpoint("model.pth")
    artifact = wandb.Artifact('model', type='model')
    artifact.add_file('model.pth')
    run.log_artifact(artifact)

wandb: Currently logged in as: armandpl (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20210524_142058-3mb1w155
wandb: Syncing run lively-sound-257


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | mse   | MeanSquaredError  | 0     
1 | mae   | MeanAbsoluteError | 0     
2 | model | ResNet            | 21.3 M
--------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)


RoadRegression(
  (mse): MeanSquaredError()
  (mae): MeanAbsoluteError()
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): B

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


test epoch end


wandb: Waiting for W&B process to finish, PID 1689
wandb: Program failed with code 1.  Press ctrl-c to abort syncing.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210524_142058-3mb1w155/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20210524_142058-3mb1w155/logs/debug-internal.log
wandb: Run summary:
wandb:             valid/mse 0.021809009835124016
wandb:             valid/mae 0.10912871360778809
wandb:                 epoch 4
wandb:   trainer/global_step 119
wandb:                 _step 6
wandb:              _runtime 86
wandb:            _timestamp 1621866145
wandb:             train/mse 0.01781730353832245
wandb:             train/mae 0.10602302849292755
wandb: Run history:
wandb:             valid/mse █▄▂▂▁
wandb:             valid/mae █▄▂▂▁
wandb:                 epoch ▁▃▅▅▆██
wandb:   trainer/global_step ▁▃▃▅▆▇█
wandb:                 _step ▁▂▃▅▆▇█
wandb:              _runtime ▁▃▃▄▆▇█
wandb:            _timestamp ▁▃▃▄▆▇█
wandb:             train/mse █▁
wandb:             train/mae █▁

AttributeError: ignored